## Data

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [ ]:
!git clone https://github.com/uhh-lt/TextGraphs17-shared-task.git

Cloning into 'TextGraphs17-shared-task'...
remote: Enumerating objects: 484, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 484 (delta 3), reused 2 (delta 2), pack-reused 480
Receiving objects: 100% (484/484), 36.63 MiB | 19.18 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
data_train = pd.read_csv('TextGraphs17-shared-task/data/tsv/train.tsv', sep='\t')
data_test = pd.read_csv('TextGraphs17-shared-task/data/tsv/test.tsv', sep='\t')

data_train.head()

,sample_id,question,questionEntity,answerEntity,groundTruthAnswerEntity,answerEntityId,questionEntityId,groundTruthAnswerEntityId,correct,graph
0,0,Whst is the name of the head of state and high...,Iran,Ruhollah Khomeini's return to Iran,Office of the Supreme Leader of Iran,Q7293530,Q794,Q16045000,False,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_..."
1,1,Whst is the name of the head of state and high...,Iran,Ruhollah Khomeini's letter to Mikhail Gorbachev,Office of the Supreme Leader of Iran,Q5952984,Q794,Q16045000,False,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q417..."
2,2,Whst is the name of the head of state and high...,Iran,Ruhollah Khomeini,Office of the Supreme Leader of Iran,Q38823,Q794,Q16045000,False,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_..."
3,3,Whst is the name of the head of state and high...,Iran,Office of the Supreme Leader of Iran,Office of the Supreme Leader of Iran,Q16045000,Q794,Q16045000,True,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_..."
4,4,Whst is the name of the head of state and high...,Iran,Mohammad Reza Pahlavi and Soraya,Office of the Supreme Leader of Iran,Q63195813,Q794,Q16045000,False,"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_..."


In [ ]:
data_test.head(20)

,sample_id,question,questionEntity,answerEntity,questionEntityId,answerEntityId,graph
0,0,"After publishing A Time to Kill, which book di...",A Time to Kill,A Clash of Kings,Q1213715,Q300370,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
1,1,"After publishing A Time to Kill, which book di...",A Time to Kill,A Feast for Crows,Q1213715,Q1764445,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
2,2,"After publishing A Time to Kill, which book di...",A Time to Kill,Fear and Loathing in Las Vegas,Q1213715,Q772435,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
3,3,"After publishing A Time to Kill, which book di...",A Time to Kill,In Cold Blood,Q1213715,Q1142887,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
4,4,"After publishing A Time to Kill, which book di...",A Time to Kill,Into the Woods,Q1213715,Q1118244,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
5,5,"After publishing A Time to Kill, which book di...",A Time to Kill,Kongenes kamp,Q1213715,Q19377881,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
6,6,"After publishing A Time to Kill, which book di...",A Time to Kill,No Country for Old Men,Q1213715,Q611689,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
7,7,"After publishing A Time to Kill, which book di...",A Time to Kill,No Country for Old Men,Q1213715,Q60411383,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q186..."
8,8,"After publishing A Time to Kill, which book di...",A Time to Kill,Slaughterhouse-Five,Q1213715,Q265954,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."
9,9,"After publishing A Time to Kill, which book di...",A Time to Kill,The Firm,Q1213715,Q1212467,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'..."


In [ ]:
N = len(set(data_train['questionEntity']) & set(data_test['questionEntity']))
train = len(set(data_train['questionEntity']))
test = len(set(data_test['questionEntity']))

f'Intersection is {N}, while train_len = {train} and test_len = {test}'

'Intersection is 173, while train_len = 2366 and test_len = 815'

In [ ]:
data_train['graph'] = data_train['graph'].apply(eval)
data_test['graph'] = data_test['graph'].apply(eval)

# LLM

### Imports

In [ ]:
import os

from tqdm import tqdm
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_train['texts'] = data_train['question'] + ' ' + data_train['answerEntity']
data_test['texts'] = data_test['question'] + ' ' + data_test['answerEntity']

In [ ]:
data_train['labels'] = data_train['correct'].astype(float)

In [ ]:
max_length = 0
for text in tqdm(data_train['texts']):
    tokenized_text = tokenizer(text, return_tensors='pt')
    input_ids = tokenized_text['input_ids']
    if input_ids.shape[1] > max_length:
        max_length = input_ids.shape[1]

print("\nMaximum tokenized text length:", max_length)

  0%|          | 0/37672 [00:00<?, ?it/s]


Maximum tokenized text length: 89


### Dataset

In [ ]:
class MyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
BATCH_SIZE = 64
max_length = 100

dataset = MyDataset(data_train['texts'], data_train['labels'], tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

### Model

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert_model = AutoModel.from_pretrained(model_name)

for param in distilbert_model.parameters():
    param.requires_grad = False


class CustomDistilBERT(nn.Module):
    def __init__(self, distilbert_model):
        super(CustomDistilBERT, self).__init__()
        self.distilbert = distilbert_model
        self.hidden_size = distilbert_model.config.hidden_size
        self.linear1 = nn.Linear(self.hidden_size, self.hidden_size * 4)
        self.linear2 = nn.Linear(self.hidden_size * 4, 1)
        self.activ = nn.ReLU()

    def forward(self, input_ids, attention_mask=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state  # Access the last hidden states
        pooled_output = last_hidden_state[:, 0, :]  # Take the [CLS] token representation
        logits = self.linear2(self.activ(self.linear1(pooled_output)))
        return logits

model = CustomDistilBERT(distilbert_model)

for param in model.linear1.parameters():
    param.requires_grad = True
for param in model.linear2.parameters():
    param.requires_grad = True

print("Trainable parameters in the linear layer:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Trainable parameters in the linear layer:
linear1.weight
linear1.bias
linear2.weight
linear2.bias


In [ ]:
model.to(device)

CustomDistilBERT(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

### Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_dir = "drive/MyDrive/Deep Learning NLP/saved_models/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
N_EPOCHS = 100
LEARNING_RATE = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
model.train()
loss_list = []
for epoch in range(N_EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(outputs.squeeze(), labels.float())

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(dataloader)
    loss_list.append(average_loss)
    print(f'Epoch {epoch + 1}/{N_EPOCHS}, Loss: {average_loss:.4f}')

    if epoch % 10 == 0:
        model_save_path = os.path.join(save_dir, f"model_epoch_{epoch + 1}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss_history': loss_list,
        }, model_save_path)

  0%|          | 0/589 [00:00<?, ?it/s]

Epoch 1/100, Loss: 26.4781


  0%|          | 0/589 [00:00<?, ?it/s]

Epoch 2/100, Loss: 26.2979


  0%|          | 0/589 [00:00<?, ?it/s]

...

Epoch 98/100, Loss: 25.3774


  0%|          | 0/589 [00:00<?, ?it/s]

Epoch 99/100, Loss: 25.3819


  0%|          | 0/589 [00:00<?, ?it/s]

Epoch 100/100, Loss: 25.3774


### Inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_save_path = 'drive/MyDrive/Deep Learning NLP/saved_models/model_epoch_99.pt'

checkpoint = torch.load(model_save_path, map_location=torch.device('cpu'))

model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

In [ ]:
data_test.head(10)

,sample_id,question,questionEntity,answerEntity,questionEntityId,answerEntityId,graph,texts
0,0,"After publishing A Time to Kill, which book di...",A Time to Kill,A Clash of Kings,Q1213715,Q300370,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
1,1,"After publishing A Time to Kill, which book di...",A Time to Kill,A Feast for Crows,Q1213715,Q1764445,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
2,2,"After publishing A Time to Kill, which book di...",A Time to Kill,Fear and Loathing in Las Vegas,Q1213715,Q772435,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
3,3,"After publishing A Time to Kill, which book di...",A Time to Kill,In Cold Blood,Q1213715,Q1142887,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
4,4,"After publishing A Time to Kill, which book di...",A Time to Kill,Into the Woods,Q1213715,Q1118244,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
5,5,"After publishing A Time to Kill, which book di...",A Time to Kill,Kongenes kamp,Q1213715,Q19377881,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
6,6,"After publishing A Time to Kill, which book di...",A Time to Kill,No Country for Old Men,Q1213715,Q611689,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
7,7,"After publishing A Time to Kill, which book di...",A Time to Kill,No Country for Old Men,Q1213715,Q60411383,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q186...","After publishing A Time to Kill, which book di..."
8,8,"After publishing A Time to Kill, which book di...",A Time to Kill,Slaughterhouse-Five,Q1213715,Q265954,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."
9,9,"After publishing A Time to Kill, which book di...",A Time to Kill,The Firm,Q1213715,Q1212467,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di..."


In [ ]:
text = data_test['question'][0] + ' ' + data_test['answerEntity'][5]
tokenized_text = tokenizer(text)
text

'After publishing A Time to Kill, which book did its author begin working on immediately? Kongenes kamp'

In [ ]:
with torch.no_grad():
    input = torch.tensor(tokenized_text['input_ids']).unsqueeze(0)
    output = model(input)
output.item()

4.27785587310791

In [ ]:
data_test['correct'] = False

In [ ]:
for question in tqdm(data_test['question'].unique()):
    lines = data_test[data_test['question'] == question]

    candidates = torch.zeros(len(lines['answerEntity']))
    for i, answer in enumerate(lines['answerEntity']):
        candidate = question + ' ' + answer
        tokenized_text = tokenizer(candidate)

        with torch.no_grad():
          input = torch.tensor(tokenized_text['input_ids']).unsqueeze(0)
          output = model(input)

        candidates[i] = output.item()

    right_answer = list(lines['answerEntity'])[torch.argmax(candidates).item()]
    data_test.loc[(data_test['question'] == question) & (data_test['answerEntity'] == right_answer), 'correct'] = True
    print(question + ' ' + right_answer)

  0%|          | 0/1000 [00:00<?, ?it/s]

After publishing A Time to Kill, which book did its author begin working on immediately? The Firm
Among the European Union countries, which one has the largest land area? Italy
Among the Final Fantasy games, which installment achieved the highest worldwide sales? Final Fantasy
At which Academy Awards was the Leonardo DiCaprio nominated for the first time? Academy Award for Best Picture
Between Ulysses S Grant and Abe Lincoln, which one was president first? Ulysses S. Grant
Did more Americans die in World War II or the American Civil War? civil war
Directed, written, produced, and co-edited by James Cameron, in which movie Kate Winslet and Leonardo DiCaprio starred together?  Avatar
Donald John Trump's first wife was born in? England
Due to which disease did the composer of La campanella died from? tuberculosis
During WWI, which country was a part of Triple Alliance ? Germany
During what war did Spanish Empire lose Philippines? Spanish American Wars of Independence
First female Pharaoh 

In [ ]:
data_test.head()

,sample_id,question,questionEntity,answerEntity,questionEntityId,answerEntityId,graph,texts,correct
0,0,"After publishing A Time to Kill, which book di...",A Time to Kill,A Clash of Kings,Q1213715,Q300370,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di...",False
1,1,"After publishing A Time to Kill, which book di...",A Time to Kill,A Feast for Crows,Q1213715,Q1764445,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di...",False
2,2,"After publishing A Time to Kill, which book di...",A Time to Kill,Fear and Loathing in Las Vegas,Q1213715,Q772435,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di...",False
3,3,"After publishing A Time to Kill, which book di...",A Time to Kill,In Cold Blood,Q1213715,Q1142887,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di...",False
4,4,"After publishing A Time to Kill, which book di...",A Time to Kill,Into the Woods,Q1213715,Q1118244,"{'nodes': [{'type': 'INTERNAL', 'name_': 'Q30'...","After publishing A Time to Kill, which book di...",False


## Saving Results

In [ ]:
data_test['prediction'] = data_test['correct'].astype(int)

In [ ]:
test_pred_path = "test_result.tsv"
data_test[["sample_id", "prediction"]].to_csv(test_pred_path, sep='\t', index=False)